In [ ]:
!pip install requests_html
!pip install selenium
!pip install webdriver-manager
!pip install --upgrade ChromeDriver

In [9]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

import time
from tqdm import tqdm

from lxml import html
from requests_html import AsyncHTMLSession 

In [2]:
#Дополнительные
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Соберем датасет из шапок новостей ###

In [5]:
url = 'https://uznews.uz/categories/proissestviya'
path_CDM = './chromedriver'
browser = webdriver.Chrome(path_CDM)# add path
browser.get(url) # Открываем страницу

time.sleep(3)


SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")
num_pages = 0
# Начали прокрутку
while True:
    num_pages+=1
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    if num_pages >= 1: 
        break
# Конец прокрутки
        
# Вытащим нужное с каждой новости       
titles = browser.find_elements_by_class_name('title')
titles = [i.text for i in titles]
num_comments = browser.find_elements_by_xpath(".//div[@class='flex gap-4']/span[@class='inline-flex items-center text-black opacity-70 font-medium text_13 gap-2'][2]/span[1]")
num_comments = [i.text for i in num_comments]
links = browser.find_elements_by_xpath(".//h3[@class='text-xl sm:text-base font-bold text-black height_2_ellips title']/..")
links = [i.get_attribute('href') for i in links]

browser.quit()

C:\Users\Lily\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Lily\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
C:\Users\Lily\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
C:\Users\Lily\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead


In [6]:
# Соберем датасет

df = pd.DataFrame(columns=['title','num_comments','link','text','comments'])
df.title = titles
df.num_comments = num_comments
df.link = links
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         30 non-null     object
 1   num_comments  30 non-null     object
 2   link          30 non-null     object
 3   text          0 non-null      object
 4   comments      0 non-null      object
dtypes: object(5)
memory usage: 1.3+ KB


### Перейдем по каждой ссылке и добавим текст самой новости и массив комментариев ###

In [7]:
def get_news_text(tree):       
    result = []
    for body in tree.findall(".//div[@class='ce-paragraph cdx-block']"): 
        # iterate recursively over all sub-elemetns of `body`
        for node in body.iter('*'):
            # append any text if it exists
            if node.text:
                result.append(node.text.strip())
            # NEW! add `node.tail` if present:
            if node.tail:
                result.append(node.tail.strip())
    string = ' '.join(result)
    string = string.replace('  ', ' ')
    return string

  
def get_news_comments(tree):
    result = tree.xpath(".//p[@class='text-base font-normal overflow-hidden']/text()")
    return result

In [12]:
texts_arr = []
comments_arr = []

asession = AsyncHTMLSession()

for link in df.link.to_numpy():     
    r = await asession.get(link)
    await r.html.arender()
    resp=r.html.raw_html
    tree = html.fromstring(resp)
    
    texts_arr.append(get_news_text(tree))
    comments_arr.append(get_news_comments(tree))
    
df.text = texts_arr
df.comments = comments_arr

In [14]:
df.tail()

,title,num_comments,link,text,comments
25,В Фергане пропали без вести женщина и двое ее ...,0,https://uznews.uz/posts/53962,Сотрудники УЧС по Ферганской области сообщили ...,[]
26,В Харькове в автобус с эвакуируемыми узбекиста...,18,https://uznews.uz/posts/53850,"В Харькове в автобус, на котором вывозили граж...","[Спасибо gосольству нашей страны, не паникуй в..."
27,В Ташкенте Spark въехал в подземный переход ме...,8,https://uznews.uz/posts/53827,"Инцидент произошел 26 февраля, примерно в 23:...","[Коробка автомат наверное, перепутал педали. А..."
28,Опубликована запись инцидента с камеры в метро...,4,https://uznews.uz/posts/53785,"По данным Генпрокуратуры, личность мужчины уст...","[Слов нет.....скорбь!, Иеееее]"
29,В Ташкенте мужчина бросился под поезд в метро,9,https://uznews.uz/posts/53782,"Сегодня, 24 февраля, примерно в 21:00, одно н...","[Вряд ли из-за кредита,у нас ведь нет коллекто..."


In [15]:
df.to_csv('news.csv', index=False)